In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import json
from aiida import load_profile
from aiida.orm import StructureData, load_code, Float, load_node
from aiida_quantumespresso_hp.workflows.hubbard import SelfConsistentHubbardWorkChain
from aiida_quantumespresso.data.hubbard_structure import HubbardStructureData
from aiida.engine import submit
from ase.atoms import Atoms
from ase.visualize import view
from aiida.plugins import DataFactory 
from pathlib import Path
from aiida_quantumespresso.common.types import SpinType
from aiida.orm import Int
from copy import deepcopy
import os

load_profile()

In [ ]:

# ! Currently, for whatever reason, codes duplicated
# pw_code = load_code('qe-dev-pw@lumi-small')
# hp_code = load_code('qe-dev-hp@lumi-small')
pw_code = load_code(2182)
hp_code = load_code(2183)

hubbard_data = load_node(3075)
print(hubbard_data.get_ase())
print(hubbard_data.get_quantum_espresso_hubbard_card())

In [ ]:
builder = SelfConsistentHubbardWorkChain.get_builder_from_protocol(
    pw_code=pw_code,
    hp_code=hp_code,
    hubbard_structure=hubbard_data,
    protocol='moderate',
    overrides=Path(os.path.join('..', 'yaml_files', 'olivine_iurii.yaml')),
    spin_type=SpinType.COLLINEAR,
)

scf_dict = builder.scf.pw.parameters.get_dict()
relax_dict = builder.relax.base.pw.parameters.get_dict()
hubbard_dict = builder.hubbard.hp.parameters.get_dict()

print(json.dumps(scf_dict, sort_keys=False, indent=4))
print(json.dumps(relax_dict, sort_keys=False, indent=4))
print(json.dumps(hubbard_dict, sort_keys=False, indent=4))

In [ ]:
# proper_olivine_builder = deepcopy(builder)
# proper_olivine_submit = submit(proper_olivine_builder)
# proper_olivine_submit_pk = proper_olivine_submit.pk
proper_olivine_submit_pk = 4282
# proper_olivine_submit_pk = 4232
!verdi process status 4282